## Step 0: Understand the Data
Get the info of errors and missing values

In [1]:
import pandas as pd
import numpy as np
s = pd.read_csv("Public_School_Characteristics_2022-23.csv")
s_b = s.copy()
s.shape

C:\Users\kurti\AppData\Local\Temp\ipykernel_1468\1985055591.py:3: DtypeWarning: Columns (0: LSTREET2) have mixed types. Specify dtype option on import or set low_memory=False.
  s = pd.read_csv("Public_School_Characteristics_2022-23.csv")


(101390, 77)

### Pattern for each column
TOTAL = TOTMENROL + TOTFENROL
TOTAL = PK + KG + G01 + G02 + G03 + G04 + G05
      + G06 + G07 + G08
      + G09 + G10 + G11 + G12
      + UG + AE
      
TOTAL = AM + AS + HI + BL + WH + HP + TR

Lunch:
TOTFRL = FRELCH + REDLCH

STUTERATIO = TOTAL / FTE

Elementary Schools:
G09 + G10 + G11 + G12 ≈ 0

High Schools:
PK + KG + G01–G08 ≈ 0

AMALM + AMALF = AM

ASALM + ASALF = AS

BLALM + BLALF = BL

HPALM + HPALF = HP

HIALM + HIALF = HI

TRALM + TRALF = TR

WHALM + WHALF = WH

### Missing Value info

In [2]:
special_values = [-1, -2, -9, "M", "N"]
s = s.replace(r'^\s*$', np.nan, regex=True) # Find any string that is completely empty ('') or only spaces (' '), replace it with np.nan

rows_with_special = s.isin(special_values).any(axis=1).sum()
print(rows_with_special)
rows_with_missing_value = s.isna().any(axis=1).sum()
print(rows_with_missing_value)
total_missing = s.isna().sum().sum()
print(total_missing)

62134
101390
1352278


## Step 1: Data Cleaning
### 1.1 Handling missing 0's for Students Each Grade

In [3]:

cols = s.loc[:, 'PK':'AE'].columns  # all columns from PK to AE

# 1️⃣ Row-wise sum of PK to AE (ignores NaN by default)
row_sum = s[cols].sum(axis=1)

# 2️⃣ Find rows where sum equals TOTAL
mask = row_sum.eq(s['TOTAL'])

# 3️⃣ Fill NaN with 0 only for those rows
s.loc[mask, cols] = s.loc[mask, cols].fillna(0)
print(s.isna().any(axis=1).sum())
print(s.isna().sum().sum())

101244
238819


### 2.2 Removing Redundant Rows and Features

#### Dropping Rows and Features with Too less value

In [4]:
# reference for rows missing value threshold
s_b.loc[49:55,:].isna().mean(axis=1)

49    0.181818
50    0.571429
51    0.571429
52    0.571429
53    0.571429
54    0.571429
55    0.571429
dtype: float64

In [5]:
s = s[s.isna().mean(axis=1) <= 0.55]
print(s.isna().any(axis=1).sum())
print(s.isna().sum().sum())
print(s.isna().sum().sort_values(ascending=False))

99577
164969
LSTREET2          99215
LZIP4             41866
FTE                2565
STUTERATIO         1814
HPALM               941
                  ...  
SY_STATUS_TEXT        0
ULOCALE               0
NMCNTY                0
LATCOD                0
LONCOD                0
Length: 77, dtype: int64


In [6]:
s = s.loc[:, s.isna().mean() <= 0.40]
print(s.isna().any(axis=1).sum())
print(s.isna().sum().sum())
print(s.isin(special_values).any(axis=1).sum())

5403
23888
60467


In [7]:
s.to_csv("Deliverable2.csv", index=False)

### 2.3 Imputate Continuous Values

In [8]:
# NCES placeholder codes (-1, -2, -9) are non observations (missing / not applicable / low quality),
# so they shouldn't be used as real numeric values in statistics and be replaced.

placeholder_codes = [-1, -2, -9, "M", "N"]

# Replace placeholder codes with NaN (only for columns where codes appear)
for c in ["STUTERATIO", "TOTFRL", "FRELCH", "REDLCH", "DIRECTCERT"]:
    if c in s.columns:
        s[c] = s[c].replace(placeholder_codes, np.nan)

# Ensure numeric columns are numeric
numeric_fix_cols = [
    "TOTAL", "FTE", "STUTERATIO",
    "TOTFRL", "FRELCH", "REDLCH", "DIRECTCERT"
]
for c in numeric_fix_cols:
    if c in s.columns:
        s[c] = pd.to_numeric(s[c], errors="coerce")

print("Missing counts (after placeholder -> NaN):")
print(s[numeric_fix_cols].isna().sum().sort_values(ascending=False))

# TOTAL: If TOTAL is missing, rebuild from grade level columns (PK to AE) when possible.
grade_cols = [c for c in s.columns if c in list(s.loc[:, "PK":"AE"].columns)]
if "TOTAL" in s.columns and grade_cols:
    total_from_grades = s[grade_cols].sum(axis=1, min_count=1)
    missing_total = s["TOTAL"].isna()
    s.loc[missing_total, "TOTAL"] = total_from_grades[missing_total]
    print("Filled TOTAL from grade sums:", int(missing_total.sum()))

# STUTERATIO: If missing, compute using TOTAL and FTE when both exist.
# First, we check that the relationship holds for most records.
if "TOTAL" in s.columns and "FTE" in s.columns and "STUTERATIO" in s.columns:
    s["RATIO_FROM_TOTAL_FTE"] = s["TOTAL"] / s["FTE"]
    s.loc[s["FTE"] == 0, "RATIO_FROM_TOTAL_FTE"] = None
    temp = s.dropna(subset=["STUTERATIO", "RATIO_FROM_TOTAL_FTE"])
    temp["ABS_DIFF"] = (temp["STUTERATIO"] - temp["RATIO_FROM_TOTAL_FTE"]).abs()
    print("mean abs diff =", round(float(temp["ABS_DIFF"].mean()), 4))
    before = s["STUTERATIO"].isna().sum()
    s.loc[s["STUTERATIO"].isna(), "STUTERATIO"] = s["RATIO_FROM_TOTAL_FTE"]
    after = s["STUTERATIO"].isna().sum()
    print("Filled STUTERATIO values:", int(before - after))
    s = s.drop(columns=["RATIO_FROM_TOTAL_FTE"], errors="ignore")
# To verify consistency internally, we compared the STUTERATIO to the computed value TOTAL/FTE. 
# We calculated the mean absolute difference to measure the avg deviation between the two values. 
# A small mean absolute difference shows that the ratios are consistent with actual enrollment and teacher counts.

# FTE: If missing, compute using TOTAL / STUTERATIO.
if "TOTAL" in s.columns and "FTE" in s.columns and "STUTERATIO" in s.columns:
    s["FTE_FROM_RATIO"] = s["TOTAL"] / s["STUTERATIO"]
    s.loc[s["STUTERATIO"] == 0, "FTE_FROM_RATIO"] = None
    before = s["FTE"].isna().sum()
    s.loc[s["FTE"].isna(), "FTE"] = s["FTE_FROM_RATIO"]
    after = s["FTE"].isna().sum()
    print("Filled FTE values:", int(before - after))
    s = s.drop(columns=["FTE_FROM_RATIO"], errors="ignore")

# Remaining missing in continuous columns: use group wise median since less sensitive to outliers
# Group by SCHOOL_LEVEL and STABR if available because school staffing and enrollment patterns are different depending on level and state.
for c in ["STUTERATIO", "FTE"]:
    if c in s.columns:
        # If both grouping columns exist, fill by SCHOOL_LEVEL and STABR
        if "SCHOOL_LEVEL" in s.columns and "STABR" in s.columns:
            for level in s["SCHOOL_LEVEL"].dropna().unique():
                for state in s["STABR"].dropna().unique():
                    # rows in this level and this state
                    condition = (s["SCHOOL_LEVEL"] == level) & (s["STABR"] == state)
                    group_values = s.loc[condition, c]
                    # calculate median if the group has at least one proper value
                    if group_values.notna().sum() > 0:
                        group_median = group_values.median()
                        # fill only missing values inside this group
                        s.loc[condition & s[c].isna(), c] = group_median
        # Otherwise, fill it with overall median
        else:
            overall_median = s[c].median()
            s.loc[s[c].isna(), c] = overall_median
# Remaining missing values in continuous variables (STUTERATIO and FTE) were imputed using group-wise median imputation. 
# Schools were grouped by SCHOOL_LEVEL and STABR because staffing and enrollment differ across different school types and states. 
# Median was chosen instead of mean to minimize sensitivity to extreme outliers in the enrollment and staffing counters. 
# This helps to uphold differences in the structure of the dataset but preventing too much data loss from deleting rows. 
# If grouping variables were unavailable, overall median imputation was used.

print("\nMissing counts after continuous imputations:")
print(s[["TOTAL", "FTE", "STUTERATIO"]].isna().sum())

Missing counts (after placeholder -> NaN):
DIRECTCERT    49548
FRELCH        18960
REDLCH        18960
TOTFRL        13098
STUTERATIO     4383
FTE            2565
TOTAL             4
dtype: int64
Filled TOTAL from grade sums: 4
mean abs diff = 0.0063
Filled STUTERATIO values: 0
Filled FTE values: 0

Missing counts after continuous imputations:
TOTAL          4
FTE            4
STUTERATIO    55
dtype: int64


### 2.4 Imputate Discrete Values

In [9]:
import re

# Fix postal codes: if LZIP is numeric, leading zeros are lost in python. This converts them back to 5 character string.
if "LZIP" in s.columns:
    s["LZIP"] = s["LZIP"].astype("Int64").astype(str).str.zfill(5)

# Address field: if missing, fill with 'Unknown'
for c in ["LSTREET1", "PHONE"]:
    if c in s.columns:
        s[c] = s[c].fillna("Unknown")
# Missing values in categorical variables not included in the analysis such as street address and phone number were replaced 
# with the placeholder value "Unknown". These variables are descriptive identifiers and are not used for analysis. 
# Replacing missing values prevents needing to delete rows unnecessarily due to empty values and ensures consistency in the dataset.

# Free/Reduced lunch fields: these are count fields and can be 0 for schools with no participants.
# We used enrollment percentage as a consideration to avoid producing impossible counts.
if "TOTFRL" in s.columns and "TOTAL" in s.columns:
    # Create FRL_PCT (only when TOTAL > 0)
    s["FRL_PCT"] = s["TOTFRL"] / s["TOTAL"]
    s.loc[s["TOTAL"] == 0, "FRL_PCT"] = None
    # Fill missing FRL_PCT with overall median
    frl_median = s["FRL_PCT"].median()
    s.loc[s["FRL_PCT"].isna(), "FRL_PCT"] = frl_median
    # missing TOTFRL using FRL_PCT * TOTAL
    s.loc[s["TOTFRL"].isna(), "TOTFRL"] = (s["FRL_PCT"] * s["TOTAL"]).round()
    # keep TOTFRL within valid range [0, TOTAL]
    s.loc[s["TOTFRL"] < 0, "TOTFRL"] = 0
    s.loc[s["TOTFRL"] > s["TOTAL"], "TOTFRL"] = s["TOTAL"]
# if FRELCH and REDLCH exist, fill missing by splitting TOTFRL 50/50
if "FRELCH" in s.columns and "REDLCH" in s.columns and "TOTFRL" in s.columns:
    # If missing, assign half of TOTFRL to free lunch
    s.loc[s["FRELCH"].isna(), "FRELCH"] = (0.5 * s["TOTFRL"]).round()
    # Reduced lunch is whatever is left
    s.loc[s["REDLCH"].isna(), "REDLCH"] = (s["TOTFRL"] - s["FRELCH"]).round()
    # Prevent negatives
    s.loc[s["REDLCH"] < 0, "REDLCH"] = 0
# The Free/Reduced Lunch variables are count fields, so they must be not negative and not greater than total enrollment.
# To handle missing values first, we converted the total FRL count into a percentage of enrollment so that any imputed values 
# would scale properly with school size. Missing percentages were filled using the median to avoid being influenced by extreme values. 
# Then we rebuilt missing FRL counts using the percentage and total enrollment, rounding to keep whole numbers. 
# More checks made sure values stayed between 0 and total enrollment. When the free and reduced breakdown was missing,
# we used a 50/50 split to maintain consistency. These steps ensured logical modification while keeping as much data as possible.

# Ensure counts are non-negative
count_cols = ["TOTAL", "TOTFRL", "FRELCH", "REDLCH", "DIRECTCERT", "TOTMENROL", "TOTFENROL", "MEMBER"]

for c in count_cols:
    if c in s.columns:
        s[c] = pd.to_numeric(s[c], errors="coerce")
        s.loc[s[c] < 0, c] = 0

print("Done discrete repairs. Remaining missing values (top 15):")
print(s.isna().sum().sort_values(ascending=False).head(15))


Done discrete repairs. Remaining missing values (top 15):
DIRECTCERT    49548
HPALM           941
HPALF           940
AMALM           914
AMALF           912
HP              894
AM              866
BLALF           830
BLALM           827
ASALM           825
ASALF           823
BL              820
TRALM           820
TRALF           818
TR              817
dtype: int64


### 2.5 Handling Edge Cases

In [10]:
# Goal: ensure totals are internally consistent and remove/repair impossible records. Sorry Monte partially gave up on this section LOL

race_cols = ["AM","AS","BL","HI","HP","TR","WH"]
race_cols = [c for c in race_cols if c in s.columns]

# Convert to numeric
for c in race_cols:
    s[c] = pd.to_numeric(s[c], errors="coerce")

# Race counts: if ALL race columns are missing, ethnicity composition cannot be computed.
# Since our problem statement uses ethnicity composition, we drop these rows.
all_race_missing = s[race_cols].isna().all(axis=1) if len(race_cols) > 0 else pd.Series(False, index=s.index)
print("Rows with all race counts missing:", int(all_race_missing.sum()))
s = s.loc[~all_race_missing].copy()

# If some race categories are missing but the known categories already sum to TOTAL, then missing categories must be 0.
if "TOTAL" in s.columns:

    race_sum_known = s[race_cols].sum(axis=1, min_count=1)

    # Loop through each row
    race_na_any = s[race_cols].isna().any(axis=1)
    for i in s.index:
        if (race_na_any.at[i] and pd.notna(s.at[i, "TOTAL"]) and race_sum_known.at[i] == s.at[i, "TOTAL"]):
            s.loc[i, race_cols] = s.loc[i, race_cols].fillna(0)

    # If exactly ONE race category is missing and TOTAL is known, fill the missing one as the remainder.
    missing_counts = s[race_cols].isna().sum(axis=1)
    one_missing = (missing_counts == 1) & s["TOTAL"].notna()
    remainder = s["TOTAL"] - s[race_cols].sum(axis=1, min_count=1)

    # Identify which column is missing per row and fill it
    for c in race_cols:
        mask = one_missing & s[c].isna() & (remainder >= 0)
        s.loc[mask, c] = remainder[mask]

    # Any remaining negative remainder means race totals exceed TOTAL (inconsistent).
    remainder_after = s["TOTAL"] - s[race_cols].sum(axis=1, min_count=1)
    inconsistent_race = remainder_after < 0
    print("Rows where race sum exceeds TOTAL (dropped):", int(inconsistent_race.sum()))
    s = s.loc[~inconsistent_race].copy()

# Gender totals vs TOTAL: if TOTMENROL + TOTFENROL != TOTAL, we keep as is but flag for awareness.
if "TOTMENROL" in s.columns and "TOTFENROL" in s.columns and "TOTAL" in s.columns:
    condition = (s["TOTMENROL"].notna() & s["TOTFENROL"].notna() & s["TOTAL"].notna())
    # Among those rows, check if male + female does NOT equal TOTAL
    inconsistent = s.loc[condition, "TOTMENROL"] + s.loc[condition, "TOTFENROL"] != s.loc[condition, "TOTAL"]
    print("Rows where male+female != TOTAL:", int(inconsistent.sum()))
    
# Final missing snapshot
print("\nRemaining missing values (top 15):")
print(s.isna().sum().sort_values(ascending=False).head(15))


Rows with all race counts missing: 813
Rows where race sum exceeds TOTAL (dropped): 0
Rows where male+female != TOTAL: 5978

Remaining missing values (top 15):
DIRECTCERT    49527
HPALM           128
HPALF           127
AMALM           101
AMALF            99
STUTERATIO       49
HP               31
AM               27
BLALF            17
BLALM            14
ASALM            12
ASALF            10
TRALM             7
BL                6
UG                5
dtype: int64


### 2.6 Feature Engineering
The features added through the imputation

In [11]:
# Ethnicity composition as proportions (robust to different school sizes)
race_cols = ["AM","AS","BL","HI","HP","TR","WH"]
race_cols = [c for c in race_cols if c in s.columns]

for c in race_cols:
    s[c] = pd.to_numeric(s[c], errors="coerce")

if "TOTAL" in s.columns:
    for c in race_cols:
        s[f"{c}_PCT"] = np.where(s["TOTAL"] > 0, s[c] / s["TOTAL"], np.nan)

    # minority share (everything except White)
    if "WH" in s.columns:
        s["MINORITY_PCT"] = 1 - s["WH_PCT"]

# School level (ordinal encoding) to support correlation
level_map = {
    "Primary": 1,
    "Middle": 2,
    "High": 3,
    "Other": 0
}
if "SCHOOL_LEVEL" in s.columns:
    s["SCHOOL_LEVEL_CODE"] = s["SCHOOL_LEVEL"].map(level_map).fillna(0).astype(int)

print("Example engineered columns:")
engineered_cols = [c for c in s.columns if c.endswith("_PCT")] + ["MINORITY_PCT","FRL_PCT","IS_CHARTER","IS_VIRTUAL","SCHOOL_LEVEL_CODE"]
print([c for c in engineered_cols if c in s.columns][:20])


Example engineered columns:
['FRL_PCT', 'AM_PCT', 'AS_PCT', 'BL_PCT', 'HI_PCT', 'HP_PCT', 'TR_PCT', 'WH_PCT', 'MINORITY_PCT', 'MINORITY_PCT', 'FRL_PCT', 'SCHOOL_LEVEL_CODE']


### 2.7 Dropping Constant Features and Duplicates

In [12]:
# Drop fully-constant columns providing no information for model
nunique = s.nunique(dropna=False)
constant_cols = list(nunique[nunique <= 1].index)
print("Constant columns:", constant_cols)

# >= 99.5% of rows share the same value
quasi_constant_cols = []
for col in s.columns:
    top_freq = s[col].value_counts(dropna=False, normalize=True).iloc[0]
    if top_freq >= 0.995:
        quasi_constant_cols.append(col)

print(">=99.5% same:", quasi_constant_cols)

# Remove duplicate records (exact duplicates or duplicate school IDs)
before = s.shape[0]
s = s.drop_duplicates()
after = s.shape[0]
print(f"Exact duplicate rows removed: {before-after}")

if "NCESSCH" in s.columns:
    before = s.shape[0]
    s = s.drop_duplicates(subset=["NCESSCH"])
    after = s.shape[0]
    print(f"Duplicate NCESSCH rows removed: {before-after}")


Constant columns: ['SURVYEAR']
>=99.5% same: ['SURVYEAR', 'G13', 'AE']
Exact duplicate rows removed: 0
Duplicate NCESSCH rows removed: 0


### 2.8 Data Shape After Cleaning

In [15]:
print("Shape after cleaning:", s.shape)
print("Total missing values remaining:", int(s.isna().sum().sum()))


Shape after cleaning: (98910, 85)
Total missing values remaining: 72232


## Step 3: Normalization and Transformation

In [16]:
!pip install matplotlib
import matplotlib.pyplot as plt

# Transformation (log1p) for heavy-tailed count features to reduce skew

# Standardization (z-score) for continuous features used in modeling

# Select a few representative count features
count_features = [c for c in ["TOTAL","FTE","TOTFRL"] if c in s.columns]
for c in count_features:
    s[f"LOG1P_{c}"] = np.log1p(s[c])

# Standardize continuous features
cont_features = [c for c in ["STUTERATIO","FRL_PCT","MINORITY_PCT","TOTAL","FTE"] if c in s.columns]
for c in cont_features:
    mean = s[c].mean()
    std = s[c].std()
    s[f"Z_{c}"] = (s[c] - mean) / std

# Correlation analysis: which engineered variables correlate with STUTERATIO?


   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   --------------------------------- ------ 6.8/8.1 MB 38.1 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 36.0 MB/s  0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 2.3/2.3 MB 66.4 MB/s  0:00:00
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 7.0/7.0 MB 48.4 MB/s  0:00:00

   ---------------------------------------- 0/7 [pyparsing]
   ---------------------------------------- 0/7 [pyparsing]
   ----- ---------------------------------- 1/7 [pillow]
   ----- ---------------------------------- 1/7 [pillow]
   ----- ---------------------------------- 1/7 [pillow]
   ----- ---------------------------------- 1/7 [pillow]
   ----- ---------------------------------- 1/7 [pillow]
   ----- ---------------------------------- 1/7 [pillow]
   ----- ----------------------


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 3.1 Export Cleaned Dataset


In [17]:
s.to_csv("Deliverable2_somewhatcleaned.csv", index=False)
print("Saved Deliverable2_somewhatcleaned.csv")


Saved Deliverable2_cleaned_final.csv
